사업보고서로부터 전체 기업의 주요 재무지표를 수집하여 가공해보자<br>
부채비율, 영업이익증가율, 매출액증가율, 매출액 상태, ROA, ROE 계산해보기.

In [1]:
import OpenDartReader
import os
from dotenv import load_dotenv

load_dotenv()
api = os.getenv("API_KEY")
dart = OpenDartReader(api)

In [3]:
import pandas as pd

base_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(base_dir, "data", "종목정보.txt")

stock_list = pd.read_csv(file_path, encoding = "euc-kr", sep = "\t",
                         usecols=["Name", "Symbol"], dtype=str)
report = dart.finstate("005380", 2020)
display(report[["fs_nm", "account_nm", "thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]])
# this term(당기), former term(전기), before former term(전전기)

,fs_nm,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
0,연결재무제표,유동자산,"83,686,357,000,000","76,082,873,000,000","73,008,101,000,000"
1,연결재무제표,비유동자산,"125,657,855,000,000","118,429,347,000,000","107,647,651,000,000"
2,연결재무제표,자산총계,"209,344,212,000,000","194,512,220,000,000","180,655,752,000,000"
3,연결재무제표,유동부채,"59,459,528,000,000","53,314,096,000,000","49,438,414,000,000"
4,연결재무제표,비유동부채,"73,543,714,000,000","64,832,370,000,000","57,321,328,000,000"
5,연결재무제표,부채총계,"133,003,242,000,000","118,146,466,000,000","106,759,742,000,000"
6,연결재무제표,자본금,"1,488,993,000,000","1,488,993,000,000","1,488,993,000,000"
7,연결재무제표,이익잉여금,"68,911,800,000,000","68,249,633,000,000","66,490,082,000,000"
8,연결재무제표,자본총계,"76,340,970,000,000","76,365,754,000,000","73,896,010,000,000"
9,연결재무제표,매출액,"103,997,601,000,000","105,746,422,000,000","96,812,609,000,000"


In [6]:
import numpy as np
import pandas as pd

def find_fins_ind_list(stock_code, stock_name, year, ind_list):
    try: # 데이터 가져오기
        report = None
        report = dart.finstate(stock_code, year)
    except:
        pass

    if report is None: 
        # 리포트가 없으면 당기, 전기, 전전기 값을 모두 제거
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 2] + [np.nan] * len(ind_list))

    else: 
        report = report[report["account_nm"].isin(ind_list)] # 관련 지표로 필터링
        if sum(report["fs_nm"] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표 사용
            report = report.loc[report["fs_nm"] == "연결재무제표"]
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표 사용
            report = report.loc[report["fs_nm"] == "재무제표"]
        data = []
        for y, c in zip([year, year - 1, year - 2],
                        ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
            record = [stock_name, y]
            for ind in ind_list:
                # acccount_nm이 indic인 행의 c 칼럼 값을 가져옴
                if sum(report["account_nm"] == ind) > 0:
                    value = report.loc[report["account_nm"] == ind, c].iloc[0]
                else:
                    value = np.nan
                record.append(value)
            data.append(record)

        return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)

In [9]:
ind_list = ["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"]
display(find_fins_ind_list("105560", "KB금융", 2020, ind_list))

{'status': '013', 'message': '조회된 데이타가 없습니다.'}



KeyError: 'account_nm'